In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [58]:
def lda_pipeline(gsize, n_comp, stem_or_lemma):
   

    comment_df = pd.read_csv(f"..\data\{stem_or_lemma}\group_{gsize[0]}_{gsize[1]}.csv") # "..\data\stem\group_15_40.csv"

    tf_idf_sk = TfidfVectorizer(norm="l1")
    X = tf_idf_sk.fit_transform(comment_df.text)


    lda = LatentDirichletAllocation(n_components=n_comp, random_state=99)
    lda.fit(X)


    encoding_matrix2 = pd.DataFrame(lda.components_, 
                                index = [f"topic{i}" for i in range(n_comp)], 
                                columns = tf_idf_sk.get_feature_names())

    top_by_topic2 = pd.DataFrame()
    for col in encoding_matrix2.index:
        topic_temp_df2 = pd.DataFrame(encoding_matrix2.T.sort_values(col, axis = "rows", ascending = False, 
                                                                key = np.abs)[col][:30]).reset_index()
        topic_temp_df2.columns = [col, col+"_value"]
        top_by_topic2 = pd.concat([top_by_topic2, topic_temp_df2.round(2)], axis = 1)

    return top_by_topic2



In [61]:
groupings = [ (7,10), (10,20), (20,40), (40,60), (60,100), (15,40), (40,100)]
n_comps = [6, 10, 15, 20]
stem_or_lemma = "lemma"



for gsize in groupings:
    writer = pd.ExcelWriter(fr'..\out\lda_{gsize}_{stem_or_lemma}.xlsx', engine='xlsxwriter')
    
    for n_comp in n_comps:
        df = lda_pipeline(gsize, n_comp, stem_or_lemma)
        df.to_excel(writer, sheet_name = f"ncomp_{n_comp}" )
    
    writer.save()


# **

In [1]:
"""comment_df = pd.read_excel("..\data\cleaned_swed_lemma.xlsx",
                            )
comment_df.drop(index = [9, 1555, 2474, 2483, 4346, 7212], inplace = True)

#remove short words
comment_df.text = comment_df.text.map(lambda line: " ".join( [word for word in line.split(" ") if (len(word) > 3 or word == "mzp")] ))"""

NameError: name 'pd' is not defined

In [37]:
#comment_df[comment_df.author.str.contains("bot", case = False)]

,Unnamed: 0,Unnamed: 0.1,comment_id,text,author,upvotes,timestamp,post_id,lens
880,880,1726,hg8o3ns,looks like posted link these should load faste...,AmputatorBot,9,1633967331,q5yryi,49
952,952,1877,hg88pze,posztodat töröltük átlépted napi korlátot posz...,rhuModBot,2,1633960547,q5we21,22
1020,1020,1993,hg7u2v8,posztodat töröltük átlépted napi korlátot posz...,rhuModBot,1,1633952137,q5tx58,22
1435,1435,2777,hg09axq,looks like posted link these should load faste...,AmputatorBot,2,1633802545,q4qiny,61
1633,1633,3104,hfwc6gt,posztodat töröltük átlépted napi korlátot posz...,rhuModBot,1,1633725082,q465u6,22
2059,2059,3999,hfrc1ys,posztodat töröltük átlépted napi korlátot posz...,rhuModBot,1,1633631250,q3fcxo,22
2210,2210,4362,hfpt4wn,ovodásoknak érik propaganda járványügyi oldal ...,ShampooBottle493,12,1633606327,q36gdf,37
2513,2513,4883,hfm1j71,posztodat töröltük átlépted napi korlátot posz...,rhuModBot,1,1633532973,q2mfd0,22
3110,3110,6001,hfa0ae4,posztodat töröltük átlépted napi korlátot posz...,rhuModBot,1,1633297636,q0rlby,22
3201,3201,6180,hf81gqc,androidon úszni reklámot vanced firefoxban ugy...,oliviaisarobot,10,1633267430,q0hgvr,146


In [19]:
comment_df.drop(index = comment_df[comment_df.author.str.contains("bot", case = False)].index, inplace = True)
comment_df.drop(index = comment_df[comment_df.text.str.contains("spam", case = False)].index, inplace = True)

In [20]:


#remove bot text


tf_idf_sk = TfidfVectorizer(norm="l1")
X = tf_idf_sk.fit_transform(comment_df.text)


In [21]:
from sklearn.decomposition import LatentDirichletAllocation

n_comp = 10

lda = lda = LatentDirichletAllocation(n_components=n_comp, random_state=99)
lda.fit(X)


LatentDirichletAllocation(random_state=99)

In [22]:
encoding_matrix2 = pd.DataFrame(lda.components_, 
                               index = [f"topic{i}" for i in range(n_comp)], 
                               columns = tf_idf_sk.get_feature_names())

top_by_topic2 = pd.DataFrame()
for col in encoding_matrix2.index:
    topic_temp_df2 = pd.DataFrame(encoding_matrix2.T.sort_values(col, axis = "rows", ascending = False, 
                                                               key = np.abs)[col][:20]).reset_index()
    topic_temp_df2.columns = [col, col+"_value"]
    top_by_topic2 = pd.concat([top_by_topic2, topic_temp_df2.round(2)], axis = 1)

top_by_topic2

C:\Users\balin\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,topic0,topic0_value,topic1,topic1_value,topic2,topic2_value,topic3,topic3_value,topic4,topic4_value,topic5,topic5_value,topic6,topic6_value,topic7,topic7_value,topic8,topic8_value,topic9,topic9_value
0,szavaz,1.42,mzp,1.57,mzp,1.73,fidesz,1.11,fidesz,1.06,gyurcsány,1.77,mzp,1.19,karácsony,1.51,mzp,1.55,szavaz,1.42
1,fidesz,1.10,szavaz,1.46,szavaz,1.53,mzp,1.06,mzp,1.02,szavaz,1.35,mond,1.04,mzp,1.37,fidesz,1.35,mzp,1.26
2,mzp,1.09,ember,1.41,hisz,1.18,szavaz,1.05,szavaz,0.96,mzp,1.34,szavaz,1.04,fidesz,1.10,dobrev,1.30,fidesz,1.01
3,karácsony,0.91,karácsony,1.30,fidesz,1.14,mond,1.00,ellenzéki,0.84,fidesz,1.30,gyurcsány,0.91,szavaz,1.09,akar,1.17,dobrev,0.95
4,akar,0.91,fidesz,1.12,mond,1.08,megy,0.94,akar,0.80,karácsony,0.97,dobrev,0.76,ember,1.04,karácsony,1.14,karácsony,0.91
5,dobrev,0.88,mond,1.04,dobrev,0.95,ország,0.78,annyi,0.79,előválasztás,0.97,karácsony,0.70,dk,0.88,nyer,1.10,ország,0.85
6,nyer,0.82,orbán,0.95,orbán,0.84,dk,0.73,megy,0.76,mond,0.85,fidesz,0.68,dobrev,0.86,szavaz,1.01,nyer,0.82
7,választás,0.77,szeptember,0.86,karácsony,0.83,rész,0.72,karácsony,0.75,dobrev,0.83,előválasztás,0.65,annyi,0.85,mond,0.98,orbán,0.81
8,ember,0.76,akar,0.84,ember,0.83,ellenzék,0.72,mond,0.70,gondol,0.81,ember,0.64,mond,0.84,dolog,0.97,mond,0.80
9,gondol,0.76,dobrev,0.83,nyer,0.80,akar,0.71,ember,0.67,ember,0.79,akar,0.63,megy,0.83,megy,0.94,ember,0.75


In [16]:
encoding_matrix2

,30,aaaaaaaaaa,aaaaaaaaaaah,aacgtatgggcctatatttgcccgacatagggcggcctaattgattgacatagcgtatgggcctatgacataggcatagggcttatgaccaaggttaag,aacgtatgggcctatatttgcccgacataggggggcctaattgattgacatagcgtatgggcctatgacataggcatagggcttatgaccaaggttaag,abbahagy,abbol,abdlocker,ablak,ablakom,...,őseim,őseink,őskövületet,ősszocialisták,őszi,őszinte,őszöd,őszödi,őőőő,űzi
topic0,0.200258,0.200136,0.200149,0.260801,0.260801,0.200262,0.200400,0.249606,0.200128,0.200127,...,0.200132,0.200139,0.200133,0.200133,0.200269,0.272321,0.200136,0.309663,0.200123,0.200108
topic1,0.227607,0.200135,0.200143,0.200130,0.200130,0.200265,0.291886,0.200140,0.200132,0.269917,...,0.229910,0.251216,0.200134,0.200139,0.200263,0.478208,0.277719,0.240829,0.200124,0.200112
topic2,0.200252,0.262197,0.200136,0.200128,0.200128,0.277965,0.258105,0.200138,0.293796,0.200125,...,0.200147,0.200168,0.200138,0.264975,0.237691,0.384510,0.200150,0.270285,0.200126,0.200111
topic3,0.200256,0.200138,0.200142,0.200130,0.200130,0.200268,0.270200,0.200138,0.200133,0.200132,...,0.200151,0.200146,0.285464,0.200140,0.200260,0.231855,0.200141,0.200433,0.200120,0.224757
topic4,0.239299,0.200137,0.266928,0.200132,0.200132,0.253568,0.200398,0.200146,0.200133,0.200140,...,0.200153,0.200149,0.200134,0.200135,0.266695,0.435747,0.200146,0.200399,0.232648,0.200114


In [39]:
#top_by_topic2.to_excel("..\out\lda_first_try.xlsx")

,topic0,topic0_value,topic1,topic1_value,topic2,topic2_value,topic3,topic3_value,topic4,topic4_value
0,poll,2.03,mar,1.20,miert,0.78,tud,37.73,mar,0.92
1,peter,1.55,há,0.74,here,0.76,mzp,34.31,bizonyiték,0.80
2,tobb,1.03,fizetsz,0.73,still,0.73,szavaz,31.81,source,0.74
3,mar,1.00,hazudnak,0.73,how,0.72,fog,30.62,jo,0.73
4,miert,0.88,eleg,0.69,can,0.71,fidesz,29.68,change,0.72
5,jelolt,0.85,biblia,0.68,sajat,0.64,dobrev,25.34,gyurcsany,0.72
6,my,0.67,unicum,0.64,szemelyesen,0.61,karácsony,23.91,szol,0.72
7,pls,0.66,pardon,0.62,bernie,0.60,mond,23.15,miert,0.70
8,with,0.64,miert,0.58,tudás,0.60,ember,22.59,trust,0.68
9,miniszterelnok,0.62,elutasitotta,0.57,win,0.59,akar,19.75,ruha,0.64


In [24]:
lda.components_.shape

(10, 42016)